In [35]:
import pandas as pd
import numpy as np
from typing import List

In [63]:
pd.set_option('mode.chained_assignment',  None) 
pd.options.display.float_format = '{:.3f}'.format

소수점 표시를 위한 출력 설정 변경

# 데이터 불러오기

In [39]:
data = []
names = ['bitcoin.csv', 'repl.csv', 'etherium.csv']

for name in names:
    data.append(pd.read_csv(name))

# 결과 데이터를 저장할 데이터 프레임 생성

In [40]:
res = [] # 결과 데이터 저장

for ticker_data in data:
    res.append(ticker_data[['close', 'volume']])

res는 순서대로 bitcoin, reple, etherium의 기술지표를 저장할 리스트

In [41]:
time = data[0][['time']] # 시간을 저장할 데이터, 시간 데이터는 별도의 독립변수로 사용되지 않을 예정

In [46]:
res[0]

,close,volume
0,2996000.000,11139.871
1,3073000.000,8279.465
2,3119000.000,10731.677
3,3096000.000,9899.323
4,3083000.000,9472.610
...,...,...
4016,22320000.000,1666.680
4017,22403000.000,1587.818
4018,23004000.000,1675.061
4019,22790000.000,1505.955


In [68]:
class Indicator(object):
    def __init__(self,
                 data: List[pd.DataFrame],
                 res: List[pd.DataFrame]):
        self.data = data
        self.res = res
    
    def get_log_scale(self) -> None:
        for i in range(3):
            self.res[i].loc[:, 'close_log'] = np.log(self.data[i]['close'])
    
    def get_nvi(self) -> None:
        def nvi(res: pd.DataFrame, coin: pd.DataFrame) -> None:
            nvi = [0] * len(coin)
            nvi[0] = 1
            close = coin['close'].copy()
            volume = coin['volume'].copy()
            
            for i in range(1, len(coin)):
                if volume[i] < volume[i-1]:
                    nvi[i] = nvi[i-1] + (close[i] - close[i-1]) * \
                            nvi[i-1] / close[i-1]
                else:
                    nvi[i] = nvi[i-1]
            res['nvi'] = nvi
        
        for i in range(3):
            nvi(self.res[i], self.data[i])
            
    def get_pvi(self) -> None:
        def pvi(res: pd.DataFrame, coin: pd.DataFrame) -> None:
            pvi = [0] * len(coin)
            pvi[0] = 100
            volume = coin['volume'].copy()
            
            for i in range(1, len(coin)):
                if volume[i] > volume[i-1]:
                    pvi[i] = pvi[i-1] + (close[i] - close[i-1] / \
                                close[i-1] * pvi[i-1])
                else:
                    pvi[i] = pvi[i-1]
            res.loc[:, 'pvi'] = pvi
        
        for i in range(3):
            pvi(self.res[i], self.data[i])
            
    

In [65]:
indicator = Indicator(data, res)

In [66]:
indicator.get_nvi()

In [67]:
res

[            close    volume    nvi
 0     2996000.000 11139.871  1.000
 1     3073000.000  8279.465  1.026
 2     3119000.000 10731.677  1.026
 3     3096000.000  9899.323  1.018
 4     3083000.000  9472.610  1.014
 ...           ...       ...    ...
 4016 22320000.000  1666.680 19.408
 4017 22403000.000  1587.818 19.480
 4018 23004000.000  1675.061 19.480
 4019 22790000.000  1505.955 19.299
 4020 23097000.000  1797.303 19.299
 
 [4021 rows x 3 columns],
        close        volume   nvi
 0    330.000         7.111 1.000
 1    358.000       509.926 1.000
 2    570.000  40394189.861 1.000
 3    434.000 116033053.156 1.000
 4    374.000  78748799.497 0.862
 ...      ...           ...   ...
 4016 528.800  51434464.678 0.670
 4017 534.600  44094548.049 0.677
 4018 548.100  46041013.658 0.677
 4019 543.400  35095065.973 0.671
 4020 542.200  47936089.691 0.671
 
 [4021 rows x 3 columns],
            close     volume    nvi
 0     310450.000 441428.101  1.000
 1     303200.000 175193.541  0.

# NVI
$$
NVI = NVI_{prev} + \frac{close - close_{prev}} {close_prev} \times NVI_{prev}
$$

In [ ]:
def NVI

# PVI
$$
PVI = PVI_{prev} + \frac{close}{close_prev} \times 100
$$

In [ ]:
def 